In [1]:
from mpetools import IslandTime, TimeSeriesPreProcess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pytz
from scipy import stats
import cv2
import numpy.ma as ma
from shapely.geometry import Polygon, LineString, Point
import os
import zipfile
from scipy.signal import savgol_filter
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import pymannkendall as mk
import statsmodels.api as sm
import tsfresh
import shapely
import pickle
from osgeo import gdal
from coastsatmaster.coastsat import SDS_preprocess, SDS_tools
import matplotlib
%matplotlib qt
from skimage.filters import threshold_multiotsu
# image processing modules
import skimage.filters as filters
import skimage.measure as measure
import skimage.morphology as morphology
import skimage.metrics as metrics
from celluloid import Camera
from statsmodels.tsa.seasonal import STL, seasonal_decompose

%load_ext autoreload

%autoreload 2

In [6]:
for yr in range(2013, 2023):
    dr = pd.date_range(datetime.datetime(yr, 3, 31), datetime.datetime(yr, 10, 1), freq='M')
    df_dr = pd.DataFrame(index=dr)
    
    if yr == 2013:
        df_dr_t = df_dr
    
    else:
        df_dr_t = pd.concat([df_dr_t, df_dr])

df_dr_t.index = [pytz.utc.localize(df_dr_t.index[i]) for i in range(len(df_dr_t.index))]

In [8]:
for i in range(50):
    try:
        plt.figure()
        #ts = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_{}_waterline'.format(i)]['monthly']['coastline_position_transect_{}_waterline'.format(i)]
        ts = island_info['timeseries_preprocessing']['df_coastline_timeseries']['coastline_position_transect_{}_waterline'.format(i)].dropna()
        ts.plot()
        condition = ts.dropna().index.isin(df_dr_t.index)
        mask = pd.Series(condition, index=ts.dropna().index)
        masked_df = ts.dropna().where(mask, np.nan)
        other_df = ts.dropna().where(~mask, np.nan)
        
        decomp = STL(ts.dropna(), period=12).fit()
        decomp.trend.plot()

        for yr in range(2013, 2023):
            if yr == 2013:
                plt.fill_between(pd.date_range(datetime.datetime(yr, 4, 1), datetime.datetime(yr, 9, 1), freq='M'), min(ts), max(ts), color='orange', alpha=0.2, label='Indian Monsoon')
            else:
                plt.fill_between(pd.date_range(datetime.datetime(yr, 4, 1), datetime.datetime(yr, 9, 1), freq='M'), min(ts), max(ts), color='orange', alpha=0.2)
        plt.title('Transect {}'.format(i))
        plt.show()
        plt.figure()
        t_statistic, p_value = stats.ttest_ind((masked_df-decomp.trend).dropna(), (other_df-decomp.trend).dropna())
        plt.boxplot([(masked_df-decomp.trend).dropna(), (other_df-decomp.trend).dropna()])
        if p_value < 0.05:
            plt.title('Transect {} - p-value: {} - significant difference'.format(i, p_value))
        else:
            plt.title('Transect {} - p-value: {} - no significant difference'.format(i, p_value))
        plt.show()
    except: continue


In [215]:
contour_counts = []

for value in to:
    binary_image = wimg_masked >= value
    labeled_image, num_features = measure.label(binary_image, connectivity=2, return_num=True)
    contours = measure.find_contours(binary_image, 0.5)  # Adjust the contour level as needed

    contour_counts.append(len(contours))

    print(f'Contours at intensity value {value}: {len(contours)}')

    # Optional: Display contours on the image
    plt.figure(figsize=(8, 8))
    plt.imshow(wimg_masked, cmap='gray')

    for contour in contours:
        plt.plot(contour[:, 1], contour[:, 0], linewidth=2)

    plt.title(f'Contours at intensity value {value}')
    plt.axis('off')
    plt.show()

print('Contour counts for all specified intensity values:', contour_counts)

Contours at intensity value -0.8841328290870489: 11
Contours at intensity value -0.7748241772823778: 4
Contours at intensity value -0.47313229830148623: 3
Contour counts for all specified intensity values: [11, 4, 3]


In [206]:
plt.plot(ts['coastline_position_transect_8'].index, ts['coastline_position_transect_8'].values, color='k')
plt.plot(ts['coastline_position_transect_8'].index, result.trend, color='r', label='trend analysis')

'''
# Indian Monsoon
for yr in range(2013, 2023):
    if yr == 2013:
        plt.fill_between(pd.date_range(datetime.datetime(yr, 4, 1), datetime.datetime(yr, 9, 1), freq='M'), min(ts['coastline_position_transect_8']), max(ts['coastline_position_transect_8']), color='orange', alpha=0.2, label='Indian Monsoon')
    else:
        plt.fill_between(pd.date_range(datetime.datetime(yr, 4, 1), datetime.datetime(yr, 9, 1), freq='M'), min(ts['coastline_position_transect_8']), max(ts['coastline_position_transect_8']), color='orange', alpha=0.2)
 '''
# El Nino
#plt.fill_between(pd.date_range(datetime.datetime(2015, 3, 1), datetime.datetime(2016, 5, 1), freq='M'), min(ts['coastline_position_transect_8']), max(ts['coastline_position_transect_8']), color='r', alpha=0.2, label='2015/2016 El Nino')

# La Nina
#plt.fill_between(pd.date_range(datetime.datetime(2020, 8, 1), datetime.datetime(2021, 5, 1), freq='M'), min(ts['coastline_position_transect_8']), max(ts['coastline_position_transect_8']), color='b', alpha=0.2, label='2020/2021 La Nina')

# 2021 storm
plt.axvline(datetime.datetime(2021, 5, 1), color='grey', linestyle='--', label='2021 storm')

# IOD positive
plt.fill_between(pd.date_range(datetime.datetime(2015, 1, 1), datetime.datetime(2016, 4, 1), freq='M'), min(ts['coastline_position_transect_8']), max(ts['coastline_position_transect_8']), color='purple', alpha=0.2, label='positive IOD')
plt.fill_between(pd.date_range(datetime.datetime(2018, 12, 1), datetime.datetime(2020, 7, 1), freq='M'), min(ts['coastline_position_transect_8']), max(ts['coastline_position_transect_8']), color='purple', alpha=0.2)

# IOD negative
plt.fill_between(pd.date_range(datetime.datetime(2016, 4, 1), datetime.datetime(2018, 12, 1), freq='M'), min(ts['coastline_position_transect_8']), max(ts['coastline_position_transect_8']), color='green', alpha=0.2, label='negative IOD')
plt.fill_between(pd.date_range(datetime.datetime(2020, 7, 1), datetime.datetime(2022, 8, 1), freq='M'), min(ts['coastline_position_transect_8']), max(ts['coastline_position_transect_8']), color='green', alpha=0.2)

plt.xlim(datetime.datetime(2015, 1, 1), datetime.datetime(2022, 8, 1))
plt.legend()

In [ ]:
from scipy import signal
def ccf_values(series1, series2):
    p = series1
    q = series2
    p = (p - np.mean(p)) / (np.std(p) * len(p))
    q = (q - np.mean(q)) / (np.std(q))  
    c = np.correlate(p, q, 'full')
    return c
    
ccf_ielts = ccf_values(ts['wave_energy_of_combined_wind_waves_and_swell'], ts['coastline_position_transect_8'])
ccf_ielts

In [ ]:
import statsmodels
statsmodels.tsa.stattools.grangercausalitytests(ts[['wave_energy_of_combined_wind_waves_and_swell', 'coastline_position_transect_8']], maxlag=24)

In [92]:
lags = signal.correlation_lags(len(ts['wave_energy_of_combined_wind_waves_and_swell']), len(ts['coastline_position_transect_8']))

def ccf_plot(lags, ccf):
    fig, ax =plt.subplots(figsize=(9, 6))
    ax.plot(lags, ccf)
    ax.axhline(-2/np.sqrt(23), color='red', label='5% confidence interval')
    ax.axhline(2/np.sqrt(23), color='red')
    ax.axvline(x = 0, color = 'black', lw = 1)
    ax.axhline(y = 0, color = 'black', lw = 1)
    ax.axhline(y = np.max(ccf), color = 'blue', lw = 1, 
    linestyle='--', label = 'highest +/- correlation')
    ax.axhline(y = np.min(ccf), color = 'blue', lw = 1, 
    linestyle='--')
    ax.set(ylim = [-1, 1])
    ax.set_title('Cross Correation IElTS Search and Registeration Count', weight='bold', fontsize = 15)
    ax.set_ylabel('Correlation Coefficients', weight='bold', 
    fontsize = 12)
    ax.set_xlabel('Time Lags', weight='bold', fontsize = 12)
    plt.legend()
    
ccf_plot(lags, ccf_ielts)

In [52]:
col = 'coastline_position_transect_0'
df2[col].plot()
fig, ax = plt.subplots(figsize=(12, 8))
sm.graphics.tsa.plot_acf(df2[col], lags=25, ax=ax)
fig, ax = plt.subplots(figsize=(12, 8))
sm.graphics.tsa.plot_pacf(df2[col], lags=25, ax=ax)
print(mk.original_test(df2[col].values))

Mann_Kendall_Test(trend='no trend', h=False, p=0.42479090505531136, z=-0.7981371498910443, Tau=-0.05742821473158552, s=-230.0, var_s=82322.0, slope=-0.010519514596271762, intercept=225.1649923156301)
